# Análise Exploratória de Dados (EDA) - HR Analytics

Este notebook realiza uma análise exploratória completa do dataset de rotatividade de funcionários.

## Objetivos:
1. Compreender a distribuição das variáveis
2. Identificar padrões e correlações
3. Detectar anomalias e outliers
4. Gerar insights para modelagem preditiva

In [ ]:
# Importar bibliotecas
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from config.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, DATASET_FILE

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')
%matplotlib inline

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas importadas com sucesso!")

## 1. Carregamento e Visão Geral dos Dados

In [ ]:
# Carregar dados
df = pd.read_csv(RAW_DATA_DIR / DATASET_FILE)

print(f"Shape do dataset: {df.shape}")
print(f"\nPrimeiras linhas:")
df.head()

In [ ]:
# Informações gerais
df.info()

In [ ]:
# Estatísticas descritivas
df.describe()

In [ ]:
# Verificar valores ausentes
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

## 2. Análise da Variável Target (Attrition)

In [ ]:
# Distribuição de Attrition
attrition_counts = df['Attrition'].value_counts()
attrition_pct = df['Attrition'].value_counts(normalize=True) * 100

print("Distribuição de Attrition:")
print(f"Stayed (0): {attrition_counts[0]} ({attrition_pct[0]:.2f}%)")
print(f"Left (1): {attrition_counts[1]} ({attrition_pct[1]:.2f}%)")

# Visualização
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]])

fig.add_trace(
    go.Bar(x=['Stayed', 'Left'], y=attrition_counts.values, name='Count'),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=['Stayed', 'Left'], values=attrition_counts.values),
    row=1, col=2
)

fig.update_layout(title_text="Distribuição de Attrition", showlegend=False)
fig.show()

## 3. Análise Univariada - Features Numéricas

In [ ]:
# Features numéricas
numeric_features = ['Age', 'YearsAtCompany', 'MonthlyIncome', 'NumberOfPromotions', 'DistanceFromHome', 'CompanyTenure']

# Histogramas
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_features):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribuição de {col}')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para detectar outliers
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_features):
    axes[idx].boxplot(df[col])
    axes[idx].set_title(f'Boxplot de {col}')
    axes[idx].set_ylabel(col)

plt.tight_layout()
plt.show()

## 4. Análise Univariada - Features Categóricas

In [ ]:
# Features categóricas
categorical_features = ['Gender', 'JobRole', 'WorkLifeBalance', 'JobSatisfaction', 
                        'PerformanceRating', 'EducationLevel', 'MaritalStatus', 
                        'JobLevel', 'CompanySize', 'RemoteWork']

# Gráficos de barras
fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.ravel()

for idx, col in enumerate(categorical_features[:10]):
    value_counts = df[col].value_counts()
    axes[idx].bar(range(len(value_counts)), value_counts.values)
    axes[idx].set_title(f'Distribuição de {col}')
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
    axes[idx].set_ylabel('Frequência')

# Remover eixos extras
for idx in range(len(categorical_features[:10]), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

## 5. Análise Bivariada - Features vs Attrition

In [ ]:
# Features numéricas vs Attrition
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_features):
    df.boxplot(column=col, by='Attrition', ax=axes[idx])
    axes[idx].set_title(f'{col} vs Attrition')
    axes[idx].set_xlabel('Attrition')
    axes[idx].set_ylabel(col)

plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
# Features categóricas vs Attrition
key_categorical = ['JobSatisfaction', 'WorkLifeBalance', 'PerformanceRating', 'JobRole']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(key_categorical):
    ct = pd.crosstab(df[col], df['Attrition'], normalize='index') * 100
    ct.plot(kind='bar', stacked=False, ax=axes[idx])
    axes[idx].set_title(f'{col} vs Attrition (%)')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Percentage')
    axes[idx].legend(['Stayed', 'Left'])
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Análise de Correlação

In [ ]:
# Carregar dados processados para análise de correlação
df_processed = pd.read_csv(PROCESSED_DATA_DIR / 'employee_attrition_processed.csv')

# Selecionar apenas features numéricas
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
correlation_matrix = df_processed[numeric_cols].corr()

# Heatmap de correlação
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Matriz de Correlação - Features Numéricas')
plt.tight_layout()
plt.show()

In [ ]:
# Correlação com Attrition
attrition_corr = correlation_matrix['Attrition'].sort_values(ascending=False)

plt.figure(figsize=(10, 8))
attrition_corr.drop('Attrition').plot(kind='barh')
plt.title('Correlação das Features com Attrition')
plt.xlabel('Correlação')
plt.tight_layout()
plt.show()

print("\nTop 10 features mais correlacionadas com Attrition:")
print(attrition_corr.drop('Attrition').head(10))

## 7. Insights e Conclusões

In [ ]:
# Resumo estatístico por Attrition
print("Resumo Estatístico por Attrition:\n")
for col in numeric_features:
    print(f"\n{col}:")
    print(df.groupby('Attrition')[col].describe()[['mean', 'std', '50%']])

In [ ]:
# Taxa de attrition por categoria
print("\nTaxa de Attrition por Categoria:\n")
for col in key_categorical:
    attrition_rate = df.groupby(col)['Attrition'].mean() * 100
    print(f"\n{col}:")
    print(attrition_rate.sort_values(ascending=False))

## Principais Insights:

1. **Desbalanceamento de Classes**: O dataset apresenta desbalanceamento, com ~80% de funcionários que permaneceram e ~20% que saíram.

2. **Features Mais Relevantes**:
   - Job Satisfaction: Funcionários com baixa satisfação têm maior taxa de rotatividade
   - Work-Life Balance: Equilíbrio ruim está associado a maior attrition
   - Monthly Income: Salários mais baixos correlacionam com maior rotatividade
   - Number of Promotions: Falta de promoções aumenta o risco de saída

3. **Padrões Identificados**:
   - Funcionários jovens (18-25) têm maior taxa de rotatividade
   - Distância de casa impacta a decisão de permanecer
   - Reconhecimento baixo está fortemente associado à saída

4. **Próximos Passos**:
   - Aplicar SMOTE para balancear as classes
   - Testar múltiplos algoritmos de ML (Logistic Regression, Random Forest, XGBoost)
   - Realizar feature engineering adicional
   - Otimizar hiperparâmetros com GridSearchCV